In [ ]:
### CODE POUR LE FILTRAGE PAR CONTENU 

import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

recipes_df = pd.read_csv('../data/raw/RAW_recipes.csv', on_bad_lines='skip')
recipes_df.info()
recipes_df = recipes_df.drop(
    ['description', 'submitted', 'contributor_id', 'tags', 'nutrition', 'n_steps', 'n_ingredients', 'steps'], axis=1)
recipes_df.head()
recipes_df = recipes_df.head(10000)
recipes_df = recipes_df.drop_duplicates()
recipes_df = recipes_df.dropna()

def text_cleaning(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text

recipes_df['ingredients'] = recipes_df['ingredients'].apply(lambda x: text_cleaning(x))
recipes_df.duplicated().sum()
recipes_df.isnull().sum()
tfidf = TfidfVectorizer(stop_words='english', max_features=30000)
tfidf_matrix = tfidf.fit_transform(recipes_df['ingredients'])
tfidf_matrix.shape
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim
indices = pd.Series(recipes_df.index, index=recipes_df['name']).drop_duplicates()
indices

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 5 most similar food
    sim_scores = sim_scores[1:6]

    food_indices = [i[0] for i in sim_scores]
    similarity_scores = [i[1] * 100 for i in sim_scores]
    
    df_reco = pd.DataFrame({
        'name': recipes_df['name'].iloc[food_indices],
        'ingredients': recipes_df['ingredients'].iloc[food_indices],
        'similarity': similarity_scores
    })
    return df_reco

result = get_recommendations('killer  lasagna')
result.head(10)